In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import json

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API")
if api_key and api_key[:3] == "sk-":
    print("API Key is SET")
else:
    print("Something Went wrong")

In [ ]:
openai = OpenAI(api_key=api_key)
MODEL = "gpt-4o-mini"

In [ ]:
system_prompt = \
"""
You are a helpful ai assistant for an Airline called AirAI.
Give short, relevent and consice answers, no more than 1 sentence
Always be accurate. If you don't know the answer, say I don't know.

"""

In [ ]:
ticket_prices = {"hyderabad":"$15","vijayawada":"$25","delhi":"$10","bengaluru":"$15"}
def get_ticket_price(destination_city):
    print(f"{destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city,"No Flights Avaliable")

#### Tools

In [ ]:
price_function = {
    "name":"get_ticket_price",
    "description": 'Get the price of the ticket to the destination city. Call this whenever you need to know the price of the ticket price, for example when a customer asks about how much is a ticket to the city',
    "parameters":{
        "type":"object",
        "properties":{
            "destination_city":{
                "type":"string",
                "description":"The city that customer wants to travel to.",
            },
        },
        "required":["destination_city"],
        "additionalProperties":False,
    }
}

In [ ]:
tools = [{"type":"function","function":price_function}]

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role":"tool",
        "content":json.dumps({"destination_city":city,"price":price}),
        "tool_call_id": tool_call.id,
    }
    return response,city

In [ ]:
def chat2(message,history):
    messages = [
        {"role":"system","content":system_prompt},
    ]
    for user_message,assistant_message in history:
        messages.append({"role":"user","content":user_message})
        messages.append({"role":"assistant","content":assistant_message})
    messages.append({"role":"user","content":message})
    response = openai.chat.completions.create(
        model = MODEL,
        messages=messages,
        tools=tools
    )
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message)
        print("==="*20)
        response,city = handle_tool_call(message)
        print("Response: ",response)
        messages.append(message)
        messages.append(response)
        print(messages)
        
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        print("Messages: ",messages)
    return response.choices[0].message.content
        

In [ ]:
gr.ChatInterface(fn=chat2).launch()